## Integration with external authentication system

#### Overview

In Federated Computing systems, many participating institutions already have their own in-house authentication systems. Instead of introducing a new authentication mechanism, we need to integrate with them. Note that this integration can be site-specific; in other words, each site may be different. Site-1 can be using OLAP, site-2 can use OAuth, and a third site can use something else.

In this example, we demonstrate NVIDIA FLARE's event-based plugin-component that can be used to integrate any type of authentication/authorization mechanism, using open-source KeyCloak integration as an example.

### Setup KeyCloak

Before we start, we need to download and start the KeyCloak service. To do that, we create a [dockerfile](./examples/custom_client_side_auth_system_integration/keycloak-setup/dockerfile) and [docker-compose](./examples/custom_client_side_auth_system_integration/keycloak-setup/docker-compose.yml) file

to start cd to ```custom_client_side_auth_system_integration/keycloak-setup``` directory and run the following command


In [ ]:
%cd keycloak-setup
! docker compose up -d --build
%cd -


#* check if the keycloak is ready 
```
!docker ps
!docker logs keycloak 
```
you should see something like 

```
    Keycloak is ready!
    ...

    Setup completed!
```




You can also check if the website and login with credential
```
user = admin
password = admin123
```

In [ ]:
http://localhost:8080


### Setup KeyCloak Authentication Plugin

This integration involves two plugins: 

* At admin client during job submision, we will require the login for the given site (site-1). We also need to get the access token to pass to the job context
* one at site local, we need to plugin for job authorization


#### Set up FL Client Job Authorization Configuration

First, we need to overwrite the default local resources.json.default with resources.json to add the custom security check component

 {
      "id": "security_handler",
      "path": "keycloak_security_handler.CustomSecurityHandler"
 }

The "keycloak_security_handler.CustomSecurityHandler" is defined as 

In [ ]:
!cat security/custom/keycloak_security_handler.py

We need to save the Keycloak public key in the following file:
`/tmp/nvflare/poc/example_project/prod_00/site-1/local/site-1/local/public_key.pem` 

Use this format:

```
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAre3kQxqOfTJ7LLRwlpotw47goqSsuyFOg9Ihx5IXDMbO8HTGuGQcdDVJaYJQYphfhp2qdw+1o6qVN2yPBxwiBWju/XZQMPbCXRBu2bVDffWJVMoelLDbr3uY9hCgYgmB7qYpDdNOmxb2+xIlg/x0q+vrRRMtdd8SGicvjg0mQSEEF4a7QOSwuDnwBX8+bMOXfyB5qQJlakNVND1Bc+MjDENkHLtImVowX9XZcz8M6Ap9Eq1z2agl6lmFxTLtZroTE6IQS/dFYPVy4rZ1Zuy5cvs/3j+SYzlplH/iP3qZs8UiKrTJMmfIuLmDbP3hEAOsEmQ/M3lRxnE4wuGxvel5rwIDAQAB
-----END PUBLIC KEY-----
```

In the local/custom/resources.json config file, it contains the following additional security handler:

```
    {
      "id": "security_handler",
      "path": "keycloak_security_handler.CustomSecurityHandler"
    }
```

The CustomSecurityHandler in the custom/keycloak_security_handler.py contains the logic to validate the admin user's KeyCloak access token when the admin user submits a job, or scheduler picks up an already submitted job from the admin user. If the access token is invalid, the job will not be authorized to run.

We can do this in the following code. 

In [ ]:
# prepare poc
! echo y | nvflare poc prepare -n 2


In [ ]:

!python get_keycloak_access_token.py /tmp/nvflare/poc/example_project/prod_00/site-1/local/site-1/local/public_key.pem


In [ ]:

!python edit_site_local_resources.py site-1 /tmp/nvflare/poc/example_project/prod_00



####  Set up Admin user authentication


In the local/custom/resources.json config file, it contains the following admin event handler. the "orgs" arg provides a list of site names, and it's corresponding KeyCloak access_token URLs:

```
      {
        "id": "auth",
        "path": "admin_auth.AdminAuth",
        "args": {
          "orgs": {
            "site-a": "http://localhost:8080/realms/myrealm/protocol/openid-connect/token"
          }
        }
      }
```

The AdminAuth event handler in the custom/admin_auth.py has the logic to acquire the KeyCloak access tokens to each individual site. When the admin user submits a job, it will set the tokens in the FLContext.


In [ ]:
! mkdir -p /tmp/nvflare/poc/example_project/prod_00/admin@nvidia.com/local/
! cp -r admin/local/* /tmp/nvflare/poc/example_project/prod_00/admin@nvidia.com/local/

In [18]:
! cp -r site/local/* /tmp/nvflare/poc/example_project/prod_00/site-1/local/

In [ ]:
! tree /tmp/nvflare/poc/example_project/prod_00/admin@nvidia.com/local/

### Testing the setup 

start poc


#### Admin Console Login

When prompted, enter the email address `admin@nvidia.com`, then provide the password for the `site-1` Keycloak instance.

#### Authentication Requirements for Job Execution

With this system configured, `site-1` will only allow authenticated admin users to submit and run jobs. `site-2` does not have this additional security requirement - any admin user can submit and run jobs there.

##### Authenticated Admin Users
An admin user (`admin@nvidia.com`) who successfully authenticates with the `site-1` Keycloak system can submit and run jobs on both `site-1` and `site-2`.

##### Unauthenticated Admin Users
If any of these occur:
- Wrong password is provided
- Keycloak system is unavailable when starting the admin tool
- Authentication fails during job submission

Then:
- Jobs won't run on `site-1`
- `site-1` will display "ERROR - Authorization failed"
- Jobs can still run successfully on `site-2`
- Use `list_jobs -d JOB_ID` to view detailed job deployment information
<!-- markdown-link-check-enable -->

Let's try this out

In [ ]:
# Prepare the data

! python code/data/download.py

In [20]:
# create job config
%cd code/

! python ./fl_jobs.py

%cd -

/home/chester/projects/NVFlare/examples/tutorials/self-paced-training/part-3_security_and_privacy/chapter-6_Security_in_federated_compute_system/06.3_site_security/custom_client_side_auth_system_integration/code
job-config for fedavg is at  /tmp/nvflare/jobs/workdir/fedavg
/home/chester/projects/NVFlare/examples/tutorials/self-paced-training/part-3_security_and_privacy/chapter-6_Security_in_federated_compute_system/06.3_site_security/custom_client_side_auth_system_integration


In [ ]:
! nvflare simulator /tmp/nvflare/jobs/workdir/fedavg -w /tmp/nvflare/workspace/tmp

* Start POC without admin console

    ``` nvflare poc start -ex admin@nvidia.com```

* Start POC admin console in separate terminal

    ``` nvflare poc start -p admin@nvidia.com```